In [19]:
from PIL import Image,ImageDraw
import io
import json
from datetime import datetime
import pytz
import astral,astral.sun
import psutil
import random
from bs4 import BeautifulSoup
import os
import time
import requests
import subprocess
from pathlib import Path
from PIL import Image, ImageEnhance
import logging
import sys


In [20]:
force_change=True  # Set to false to wait
min_auto_seconds=58 # Seconds to wait

In [21]:
wallpaper_dir = os.path.expanduser("~/Pictures/Wallpapers/")

In [22]:
if not force_change:
    if((time.time()-os.stat('randomWp.log').st_mtime) < min_auto_seconds):
        sys.exit(0)

In [23]:
##Magic Logging Bootstrap
logging.basicConfig(
    filename='randomWp.log',
    level=logging.INFO,
    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

exlogger = logging.getLogger("Exception")
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return
    exlogger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [24]:
def downloadSomePapers(size=None):
    resolution = ""
    if size is not None:
        resolution = f"&atleast={size}"
    if os.path.exists('wp-downloading'):
        if((time.time() - os.stat('wp-downloading').st_mtime) > 3*60):
            os.unlink('wp-downloading')
        else:
            print(f'wp-downloading file is present and good for {(3*60)-(time.time() - os.stat("wp-downloading").st_mtime)} seconds more')
            sys.exit(1)

    with open('wp-downloading', 'w') as f:
        f.write(' ')

    sfw = 1
    sketchy = 0
    nsfw = 0
    blacklist=[
        'https://wallhaven.cc/images/layout/logo_sm.png'
    ]
    wallpaper_dir = os.path.expanduser("~/Pictures/Wallpapers")
    if not os.path.exists(wallpaper_dir):
        os.mkdir(wallpaper_dir)

    def process4WalledPage(url):
        res = requests.get(url, headers={"User-Agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'})
        soup = BeautifulSoup(res.text, features="lxml")
        for item in soup.find_all('img'):
            if 'data-src' in item.attrs:
                if item.attrs['data-src'] not in blacklist:
                    item = item.attrs['data-src']
                    item = item.replace("https://th.wallhaven.cc/small/", "")
                    item = item.replace("/", "/wallhaven-")
                    url = f"https://w.wallhaven.cc/full/{item}"
                    fn = item.split('/').pop()
                    yield((fn, url))
    random_url = f"https://wallhaven.cc/search?categories=111&purity={sfw}{sketchy}{nsfw}&atleast={resolution}&sorting=random&order=desc&page=3"
    print(f'scraping {random_url}')
    for filename, url in process4WalledPage(random_url):
        print(f"{url} => {filename}")
        res = requests.get(url)
        if res.status_code == 200 and len(res.content)>1:
            img = Image.open(io.BytesIO(res.content))
            height,width = img.size
            size_str = f'{height}x{width}'
            print(f"adding image to: {size_str}")
            size_folder = Path(os.path.join(wallpaper_dir,size_str))
            if not size_folder.exists():
                size_folder.mkdir()
            out_path = os.path.join(wallpaper_dir,size_str,filename)
            with open(out_path,'wb') as f:
                f.write(res.content)
    os.unlink('wp-downloading')

In [25]:
def get_DBUS_SESSION_BUS_ADDRESS():
    PID = None
    for p in psutil.process_iter():
        if p.name() == "gnome-session-binary":
            PID = p.pid
            break

    logging.debug(PID)

    DBUS_SESSION_BUS_ADDRESS=None
    with open(f'/proc/{PID}/environ', 'rb') as f:
        d = f.read()
        for line in d.split(b'\x00'):
            if b'DBUS_SESSION_BUS_ADDRESS' in line:
                line = line.split(b'=', maxsplit=1)
                DBUS_SESSION_BUS_ADDRESS = line[1].decode('utf-8')
    logging.debug(DBUS_SESSION_BUS_ADDRESS)
    os.environ['DBUS_SESSION_BUS_ADDRESS'] = DBUS_SESSION_BUS_ADDRESS
    return DBUS_SESSION_BUS_ADDRESS

In [26]:
def getSunset():
    latitude = 42.703750
    longitude = -84.438410
    tz_chicago = pytz.timezone('America/Chicago')
    tz_name = 'America/Chicago'
    for_date = datetime.now().date()
    locationInfoObj = astral.LocationInfo('Custom Name', 'My Region', tz_name, latitude, longitude)
    s = astral.sun.sun(locationInfoObj.observer, date=for_date)
    return s['sunset'].astimezone(tz_chicago)

In [27]:
from screeninfo import get_monitors

In [28]:
max_height = 0
max_width = 0

for m in get_monitors():
    logging.info(m)
    if m.x + m.width > max_width:
        max_width = m.x + m.width
    if m.y + m.height > max_height:
        max_height =  m.y + m.height
logging.info(f"maxes {max_width},{max_height}")

Monitor(x=3000, y=0, width=1080, height=1920, width_mm=300, height_mm=530, name='XWAYLAND0', is_primary=False)
Monitor(x=3000, y=0, width=1080, height=1920, width_mm=300, height_mm=530, name='XWAYLAND0', is_primary=False)
Monitor(x=0, y=0, width=1080, height=1920, width_mm=300, height_mm=530, name='XWAYLAND1', is_primary=False)
Monitor(x=0, y=0, width=1080, height=1920, width_mm=300, height_mm=530, name='XWAYLAND1', is_primary=False)
Monitor(x=1080, y=0, width=1920, height=1080, width_mm=530, height_mm=300, name='XWAYLAND2', is_primary=False)
Monitor(x=1080, y=0, width=1920, height=1080, width_mm=530, height_mm=300, name='XWAYLAND2', is_primary=False)
Monitor(x=1080, y=1080, width=1920, height=1080, width_mm=310, height_mm=170, name='XWAYLAND3', is_primary=True)
Monitor(x=1080, y=1080, width=1920, height=1080, width_mm=310, height_mm=170, name='XWAYLAND3', is_primary=True)
maxes 4080,2160
maxes 4080,2160


In [29]:
p = Path('weather.json')
not p.exists() or (datetime.now()-datetime.fromtimestamp(p.stat().st_mtime)).seconds > 60*60*10

False

In [30]:
def isStorming():
    if(not p.exists() or (datetime.now()-datetime.fromtimestamp(p.stat().st_mtime)).seconds > 60*60*10):
        weather = requests.get("https://dataservice.accuweather.com/forecasts/v1/hourly/1hour/2211489?apikey=lzAwADaLJIEGXGWS4eCsSL7jVGmiBw9l&language=en-us&details=true").json()
        if 'Code' in weather and weather['Code'] =='ServiceUnavailable':
            return False
        p.write_text(json.dumps(weather))
    else:
        weather = json.loads(p.read_text())
    return weather[0]['IconPhrase'] == 'Thunderstorms'

In [31]:
def imgToRatio(h,w):
    if h>w:
        return '16x9'
    elif w>h:
        return '9x16'
    else:
        return 'square'

In [32]:
def getRandomBackground(size=None,used=None):
    if used is None:
        used = []
    if size is None:
        size = random.choice(os.listdir(wallpaper_dir))
        logging.info("Selecting random size images")
    try:
        bgs = os.listdir(os.path.join(wallpaper_dir,size))
        if len(bgs)==0:
            return None
        path = os.path.join(wallpaper_dir,size,random.choice(bgs))
        max_i=10
        if path in used:
            while path in used:
                max_i-=1
                print('ReRollin', max_i)
                path = os.path.join(wallpaper_dir,size,random.choice(bgs))
                if max_i ==0:
                    return None
        return path
    except Exception as e:
        print(e)
        return None

In [33]:
class ImageSpot():
    def __init__(self,image,x,y,h,w):
        self.image = image
        self.x = x
        self.y = y
        self.height = h
        self.width = w
        print(self)
        self.fillColor = '';
        self.size_str = imgToRatio(h,w)
        self.ratio = w/h

    def fill(self, im):
        pos = (self.x, self.y)
        logging.info(f'pasting at {pos}, {im.size}')
        self.image.paste(im, pos)

    def fakeit(self):
        shape = [(self.x, self.y), (self.x+self.width, self.y + self.height)]
        # create rectangle image
        img1 = ImageDraw.Draw(self.image)
        img1.rectangle(shape, fill=self.fillColor)
    def __repr__(self):
        return f"Monitor({self.x},{self.y}) {self.width}X{self.height}"

In [34]:
im = Image.new(mode="RGBA", size=(max_width,max_height))
image_spots = []
for m in get_monitors():
    image_spots.append(ImageSpot(image=im, x=m.x, y=m.y, h=m.height, w=m.width))


backup_colors = ['#151515','#131313','#171717','#ff00ff']
needs_more_papers=[]
used = []
for item in image_spots:
    logging.info(item)
    wallpaper = None
    wallpaper = getRandomBackground(item.size_str,used)
    logging.info(f'wpf {wallpaper}')
    if wallpaper is None:
        logging.info(f"no image of size:{item.size_str}, faking it")
        needs_more_papers.append(item.size_str)
        item.fillColor = backup_colors.pop(0)
        item.fakeit()
    else:
        img = Image.open(wallpaper)
        logging.info(f"{item} - {item.size_str},{img.size},{wallpaper}")
        item.fill(img)
        used.append(wallpaper)
    print()
wallpaper = os.path.expanduser('~/bg.png')

a = getSunset()
if (datetime.now().time().hour > a.hour or isStorming()):
    print("Sunset mode")
    enhancer = ImageEnhance.Brightness(im)
    factor = 0.5  # Darkens the image
    im = enhancer.enhance(factor)

im.save(wallpaper)

Monitor(3000,0) 1080X1920
Monitor(0,0) 1080X1920
Monitor(1080,0) 1920X1080
Monitor(1080,1080) 1920X1080
Monitor(3000,0) 1080X1920
Monitor(3000,0) 1080X1920
wpf /home/srudloff/Pictures/Wallpapers/16x9/wallhaven-mpexx8.jpg
wpf /home/srudloff/Pictures/Wallpapers/16x9/wallhaven-mpexx8.jpg
Monitor(3000,0) 1080X1920 - 16x9,(1080, 1920),/home/srudloff/Pictures/Wallpapers/16x9/wallhaven-mpexx8.jpg
Monitor(3000,0) 1080X1920 - 16x9,(1080, 1920),/home/srudloff/Pictures/Wallpapers/16x9/wallhaven-mpexx8.jpg
pasting at (3000, 0), (1080, 1920)
pasting at (3000, 0), (1080, 1920)

Monitor(0,0) 1080X1920
Monitor(0,0) 1080X1920
wpf /home/srudloff/Pictures/Wallpapers/16x9/wallhaven-0wgmk7.jpg
wpf /home/srudloff/Pictures/Wallpapers/16x9/wallhaven-0wgmk7.jpg
Monitor(0,0) 1080X1920 - 16x9,(1080, 1920),/home/srudloff/Pictures/Wallpapers/16x9/wallhaven-0wgmk7.jpg
Monitor(0,0) 1080X1920 - 16x9,(1080, 1920),/home/srudloff/Pictures/Wallpapers/16x9/wallhaven-0wgmk7.jpg
pasting at (0, 0), (1080, 1920)
pasting at (0

In [35]:
get_DBUS_SESSION_BUS_ADDRESS()

'unix:path=/run/user/1000/bus'

In [36]:
cmd =f"DBUS_SESSION_BUS_ADDRESS='{get_DBUS_SESSION_BUS_ADDRESS()}' /usr/bin/gsettings set org.gnome.desktop.background picture-uri {wallpaper}"
logging.debug(cmd)
os.system(cmd)

cmd =f"DBUS_SESSION_BUS_ADDRESS='{get_DBUS_SESSION_BUS_ADDRESS()}' /usr/bin/gsettings set org.gnome.desktop.background picture-uri-dark {wallpaper}"
logging.debug(cmd)
os.system(cmd)

try:
    os.unlink('wp-lock')
except Exception:
    pass

cmd =f"DBUS_SESSION_BUS_ADDRESS='{get_DBUS_SESSION_BUS_ADDRESS()}' /usr/bin/gsettings set org.gnome.desktop.background picture-options spanned"
os.system(cmd)

0

In [18]:
urls =[
    "https://wallhaven.cc/search?categories=111&purity=100&ratios=9x16&sorting=random&order=desc",
    "https://wallhaven.cc/search?categories=111&purity=100&ratios=16x9&sorting=random&order=desc"
]


blacklist=[
    'https://wallhaven.cc/images/layout/logo_sm.png'
]

def process4WalledPage(url):
    res = requests.get(url, headers={"User-Agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'})
    soup = BeautifulSoup(res.text, features="lxml")
    for item in soup.find_all('img'):
        if 'data-src' in item.attrs:
            if item.attrs['data-src'] not in blacklist:
                item = item.attrs['data-src']
                item = item.replace("https://th.wallhaven.cc/small/", "")
                item = item.replace("/", "/wallhaven-")
                url = f"https://w.wallhaven.cc/full/{item}"
                fn = item.split('/').pop()
                yield((fn, url))
for url in urls:
    print(f"{url}")
    for filename, url in process4WalledPage(url):
        res = requests.get(url)
        if res.status_code == 200 and len(res.content)>1:
            print(f"{url} => {filename}")
            img = Image.open(io.BytesIO(res.content))
            height,width = img.size
            size_str = f'{height}x{width}'
            print(f"adding image to: {size_str}")
            size_folder = Path(os.path.join(wallpaper_dir,size_str))
            if not size_folder.exists():
                size_folder.mkdir()
            out_path = os.path.join(wallpaper_dir,size_str,filename)
            with open(out_path,'wb') as f:
                f.write(res.content)

https://wallhaven.cc/search?categories=111&purity=100&ratios=9x16&sorting=random&order=desc
https://w.wallhaven.cc/full/3k/wallhaven-3kjd16.jpg => wallhaven-3kjd16.jpg
adding image to: 1242x2208
https://w.wallhaven.cc/full/k9/wallhaven-k9yre6.jpg => wallhaven-k9yre6.jpg
adding image to: 1200x2156
https://w.wallhaven.cc/full/96/wallhaven-96e37w.jpg => wallhaven-96e37w.jpg
adding image to: 1152x2048
https://w.wallhaven.cc/full/eo/wallhaven-eoql1k.jpg => wallhaven-eoql1k.jpg
adding image to: 1080x1920
https://w.wallhaven.cc/full/d5/wallhaven-d5y9go.jpg => wallhaven-d5y9go.jpg
adding image to: 1242x2208
https://w.wallhaven.cc/full/x8/wallhaven-x8y1pz.jpg => wallhaven-x8y1pz.jpg
adding image to: 2239x3967
https://w.wallhaven.cc/full/73/wallhaven-73g6re.jpg => wallhaven-73g6re.jpg
adding image to: 1080x1920
https://w.wallhaven.cc/full/3z/wallhaven-3z95v3.jpg => wallhaven-3z95v3.jpg
adding image to: 1080x1920
https://w.wallhaven.cc/full/j8/wallhaven-j88mrp.jpg => wallhaven-j88mrp.jpg
adding i